In [ ]:
knitr::opts_chunk$set(echo = TRUE)
library(tidyverse)
library(scales)
# detach("package:rlang", unload=TRUE)


Datensatz von: Bodo Winter Thema: Politeness and Pitch (F0)

(cf. <http://www.bodowinter.com/tutorial/bw_LME_tutorial1.pdf>)

(cf. <https://bodowinter.com/tutorial/bw_LME_tutorial.pdf>)

Gliederung unserer quantitativen Analyse

1\. Laden der Datei

2\. Kennenlernen der Daten und Säubern

3\. Hypothesen

4\. Test und Ergebnisse

5\. Schluss

## 1. Datei laden


In [ ]:
# politeness <- read.csv("/cloud/project/data/politeness_data.csv")
politeness <- read.csv("data/politeness_data.csv")


## 2. Kennenlernen der Daten und Säubern

Welche Variablen enthält die Datei?


In [ ]:
head(politeness)
  


In [ ]:
glimpse(politeness)



Am Experiment nahmen 6 Versuchspersonen teil (F1, ..., M7). Von jeder
Versuchsperson (subject) haben wir 14 Messpunkte (n = 14).


In [ ]:
politeness %>% 
  count(subject)


Versuchspersonen: je 3 sind weiblich bzw. männlich.



In [ ]:
politeness %>% 
  count(subject, gender)


Pro Verhaltensweise stehen uns 42 Messpunkte zur Verfügung, um unsere (unten
folgende) Hypothese zu überprüfen.


In [ ]:
politeness %>% 
  count(attitude)


Berechnen wir mal die Grundfrequenz!



In [ ]:
politeness %>% 
  mean(frequency)


"NA": In unserer Datenreihe fehlt eine Frequenz. Wir entfernen diese Datenzeile,
um die durchschnittliche Frequenz (mean) zu berechnen.


In [ ]:
politeness %>% 
  drop_na(frequency) %>%
  summarise(av_freq = mean(frequency))


Wir haben gerade die Durchschnittsfrequenz für alle Versuchspersonen berechnet.
Berechnen wir sie nun getrennt nach weiblichen und männlichen Versuchspersonen!


In [ ]:
politeness %>% 
  drop_na(frequency) %>%
  group_by(gender) %>% 
  summarise(av_freq = mean(frequency))


Erwartungsgemäß ist der Durchschnittswert bei Frauen höher als bei Männern:
Frauen haben meist eine höhere Stimme als Männer.

Ein Blick auf die Durchschnittsfrequenzen bei höflicher und informeller
Sprechweise: In unserer Stichprobe mit 6 Versuchspersonen (je 14
Frequenzmessungen) zeigt sich ein Unterschied von etwa 18,2 Hz, und zwar 202,59
- 184,36.


In [ ]:
politeness %>% 
  drop_na() %>% 
  group_by(attitude) %>% 
  summarise(avg_freq = mean(frequency),
            sd_freq = sd(frequency))


In [ ]:
# politeness %>% 
#   drop_na %>% 
#   transmute(attitude, frequency) %>% 
#   mutate(attitude = str_replace(attitude, "pol", "1"),
#          attitude = str_replace(attitude, "inf", "0")) %>% 
#   mutate(attitude = parse_number(attitude))


## 3. Hypothesen

H0: Der durchschnittliche Grundfrequenzverlauf (F0) bei höflichem oder
informellem Sprechverhalten (attitude) ist gleich.

H1: Der durchschnittliche Grundfrequenzverlauf (F0) bei höflichem
Sprechverhalten unterscheidet sich vom informellen.

Nach unserem bisherigen Wissen erwarten wir, dass unsere Daten die Hypothese H1
bestätigen werden.

Das überprüfen wir zunächst mit einem t-Test, anschließend mit einer linearen
Regression.

## 4. Test

t-Test

Zunächst ein Blick auf die Durchschnittsfrequenzen bei höflicher und informeller
Sprechweise. In unserer Stichprobe mit 6 Versuchspersonen (je 14
Frequenzmessungen) zeigt sich ein Unterschied von etwa 18,2 Hz.

Gemäß Hypothese H1 ist der Unterschied nicht zufällig entstanden, sondern kann
auf die Gesamtpopulation deutscher Sprecher verallgemeinert werden.

Nicht so gemäß Hypothese H0: Der Mittelwertunterschied zwischen den Stichproben
kann zufällig entstanden sein, denn wenn wir eine andere Stichprobe genommen
hätten, wäre der Unterschied vielleicht gleich Null gewesen.

Mit statistischen Tests können wir diese beiden Hypothesen überprüfen. Einer
davon ist der t-Test.


In [ ]:
politeness %>% 
  drop_na() %>% 
  group_by(attitude) %>% 
  summarise(avg_freq = mean(frequency),
            sd_freq = sd(frequency))


Der t-Test bestätigt H1 nicht (p \> 0,05):



In [ ]:
t.test(frequency ~ attitude, data = politeness, paired = F, var.equal = T)



In [ ]:
pol = politeness$frequency[politeness$attitude == "pol"]
inf = politeness$frequency[politeness$attitude == "inf"]
t.test(pol, inf)


In [ ]:
polite <- politeness %>% 
  select(attitude, frequency) %>% 
  filter(attitude == "pol") %>% 
  select(-attitude)

informal <- politeness %>% 
  select(attitude, frequency) %>% 
  filter(attitude == "inf") %>% 
  select(-attitude)

t.test(polite, informal)


Die lineare Regression hat den großen Vorteil, dass man mehr als eine
unabhängige Variable (Prädiktor) verwenden kann, um die Hypothese zu testen. Wir
wählen Geschlecht (gender) und Sprechverhalten (attitude) als unabhängige
Variablen, der Grundfrequenzverlauf (frequency) ist die abhängige Variable.


In [ ]:
politeness %>% 
  drop_na %>% 
  lm(frequency ~ attitude + gender, data = .) %>% 
  summary()


Die lineare Regression bestätigt die Hypothese H1: F(2;80 = 98,38; p \< 0,001).
Die Versuchspersonen sprechen demnach in einer anderen Tonlage, wenn sie höflich
sprechen, und zwar um ca. 19,5 Hz tiefer als wenn sie informell sprechen (p =
0,0146). Außerdem bestätigt die lineare Regression (erwartungsgemäß) auch, dass
die männlichen Versuchspersonen mit einer tieferen Stimme sprechen als die
weiblichen, und zwar um durchschnittlich 108 Hz. Der R\^2-Wert beträgt 0,71
(d.h. etwa 71%). Das bedeutet, dass mit dem Regressionsergebnis ca. 71% der
Variabilität unserer Daten erklärt wird.


In [ ]:
politeness %>% 
  ggplot(aes(attitude, frequency, group = attitude, fill = attitude)) +
  geom_boxplot() +
  facet_wrap(~ gender) +
  geom_hline(yintercept = c(202.5), lty = 3) +
  geom_hline(yintercept = c(184.3), lty = 2) +
  geom_jitter(width = 0.2)


In [ ]:
m <- lm(frequency ~ attitude*gender, data = politeness)
library(effects)
allEffects(m)
plot(allEffects(m), multiline=TRUE, grid=TRUE, rug=FALSE, as.table=TRUE)


## 5. Schluss

Die Regressionsanalyse hat H1 bestätigt, d.h. die Grundfrequenz beim höflichen
Sprechen unterscheidet sich vom informellen Sprechen. Beim höflichen Sprechen
sprachen die Versuchspersonen mit einer durchschnittlich 19,5 Hz tieferen Stimme
(bei den weiblichen Versuchspersonen ca. 27 Hz, bei den männlichen mehr als 11
Hz).
